In [140]:
import os

os.environ['CUDA_VISIBLE_DEVICES']='2, 3'

In [113]:
import os
import shutil
import torch.nn as nn
from torchvision import transforms,datasets
import torch

In [120]:
class config:
    # 数据集路径
    datas_root = '/home/xtu_conda/xtj2020/data/dogs-vs-cats/'
    # 原始数据存放路径
    origin_datas = '/home/xtu_conda/xtj2020/data/dogs-vs-cats/originData/'
    # 训练集存放路径
    train_datas ='/home/xtu_conda/xtj2020/data/dogs-vs-cats/trainData/'
    # 验证集存放路径
    valid_datas = '/home/xtu_conda/xtj2020/data/dogs-vs-cats/validData/'
    # 测试集存放路径
    test_datas = '/home/xtu_conda/xtj2020/data/dogs-vs-cats/testData/'
    # 测试结果保存位置
    result_files = '/home/xtu_conda/xtj2020/data/dogs-vs-cats/result.csv'

    # 常用参数
    # batch size
    batch_size = 32
    # mean and std
    # 通过抽样计算得到图片的均值mean和标准差std
    mean = [0.470, 0.431, 0.393]
    std = [0.274, 0.263, 0.260]

    # 预训练模型路径
    path_vgg16 = './models/vgg16-397923af.pth'
conf = config()

In [121]:
#需要从原始数据中筛选出猫狗数据
origindata_files = os.listdir(os.path.join(conf.origin_datas,'train/'))
# print(origindata_files)
dogfiles = list(filter(lambda x : x[:3] == "dog",origindata_files))
catfiles = list(filter(lambda x : x[:3] == "cat",origindata_files))
dognums_train,catnums_train = len(dogfiles)*0.8,len(catfiles)*0.8
for i in range(len(dogfiles)):
    pre_path = os.path.join(conf.origin_datas,'train/',dogfiles[i])
    if i < dognums_train:
        new_path = os.path.join(conf.train_datas,'dog/')
    else:
        new_path = os.path.join(conf.valid_datas,'dog/')
    shutil.move(pre_path,new_path)
    
for i in range(len(catfiles)):
    pre_path = os.path.join(conf.origin_datas,'train/',catfiles[i])
    if i < catnums_train:
        new_path = os.path.join(conf.train_datas,'cat/')
    else:
        new_path = os.path.join(conf.valid_datas,'cat/')
    shutil.move(pre_path,new_path)

In [122]:
class Dataset:
    '''
    通过该子类实现：
    图像的预处理，加载数据集与验证集（含标签）
    两个专用方法，获取数据与获取长度
    '''
    def __init__(self, train=True):
        # 图片预处理
        # Compose用于将多个transfrom组合起来
        # ToTensor()将像素从[0, 255]转换为[0, 1.0]
        # Normalize()用均值和标准差对图像标准化处理 x'=(x-mean)/std，加速收敛
        self.transform = transforms.Compose([transforms.Resize((64, 64)),
                                             transforms.ToTensor(),
                                             transforms.Normalize(conf.mean, conf.std)])
        self.train = train
        # 加载训练数据集和验证数据集
        if train:

            # 数据加载
            # 这里使用通用的ImageFolder和DataLoader数据加载器
            # 数据类型 data_images = {'train': xxx, 'valid': xxx}
            self.data_images = {x: datasets.ImageFolder(root=os.path.join(conf.datas_root, x),
                                                        transform=self.transform)
                                for x in ['trainData', 'validData']}
            self.data_images_loader = {x: torch.utils.data.DataLoader(dataset=self.data_images[x],
                                                                      batch_size=conf.batch_size,
                                                                      shuffle=True)
                                       for x in ['trainData', 'validData']}
            # 图片分类 ['cat', 'dog']
            self.classes = self.data_images['trainData'].classes
            # 图片分类键值对 {'cat': 0, 'dog': 1}
            self.classes_index = self.data_images['trainData'].class_to_idx

        # 加载测试数据集
        else:
            images = [os.path.join(conf.data_test_root, img) for img in os.listdir(conf.data_test_root)]
            self.images = sorted(images, key=lambda x: int(x.split('.')[-2].split('/')[-1]))

    # 重载专有方法__getitem__
    def __getitem__(self, index):
        img_path = self.images[index]
        label = int(self.images[index].split('.')[-2].split('/')[-1])
        data_images_test = Image.open(img_path)
        data_images_test = self.transform(data_images_test)
        return data_images_test, label

    # 重载专有方法__len__
    def __len__(self):
        return len(self.images)

In [139]:
# 需继承torch.nn.Module类
class VGG16(nn.Module):

    def __init__(self):
        super(VGG16, self).__init__()
        # 定义卷积层和池化层，共13层卷积，5层池化
        self.conv = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            # nn.ReLU(),
            # nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            # nn.ReLU(),
            # nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2),
        )

        # 简化版全连接层
        self.classes = nn.Sequential(
            nn.Linear(4 * 4 * 512, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 2)
        )

        # VGG-16的全连接层
        # self.classes = nn.Sequential(
        #     nn.Linear(7 *7 *512, 4096),
        #     nn.ReLU(),
        #     nn.Dropout(p=0.5),
        #     nn.Linear(4096, 4096),
        #     nn.ReLU(),
        #     nn.Dropout(p=0.5),
        #     nn.Linear(4096, 2)
        # )

    # 定义每次执行的计算步骤
    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 4 * 4* 512)
        x = self.classes(x)
        return x

In [137]:
#这个函数是独立于数据迭代器的
def get_mean_std(data_images):
    '''
    :param data_images: 加载好的数据集
    :return: mean,std
    '''
    times, mean, std = 0, 0, 0
    data_loader = {x: torch.utils.data.DataLoader(dataset=data_images[x],
                                                  batch_size=1000,
                                                  shuffle=True)
                          for x in ['trainData', 'validData']}
    for imgs, labels in data_loader['trainData']:
        # imgs.shape = torch.Size([32, 3, 64, 64])
        times += 1
        mean += np.mean(imgs.numpy(), axis=(0, 2, 3))
        std += np.std(imgs.numpy(), axis=(0, 2, 3))
        print('times:', times)

    mean /= times
    std /= times
    return mean, std

dataset=Dataset()
get_mean_std(dataset.data_images)

In [136]:
data_loader = {x:torch.utils.data.DataLoader(dataset = data_im)}

times: 1
times: 2
times: 3
times: 4
times: 5
times: 6
times: 7
times: 8
times: 9
times: 10
times: 11
times: 12
times: 13
times: 14
times: 15
times: 16
times: 17
times: 18
times: 19
times: 20


(array([0.06826375, 0.09302256, 0.09314134], dtype=float32),
 array([0.9203984, 0.93364  , 0.9546887], dtype=float32))

In [126]:
dst = Dataset()
# 预览其中一个批次的数据
imgs, labels = iter(dst.data_images_loader['trainData']).next()
print(imgs.shape)
print(labels)
print(labels.shape)

torch.Size([32, 3, 64, 64])
tensor([0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 1, 0, 0, 1])
torch.Size([32])


In [128]:
classes_index = data_images['trainData'].class_to_idx
print(classes_index)

NameError: name 'data_images' is not defined

In [143]:
os.environ['CUDA_VISIBLE_DEVICES']='2, 3'
# 定义损失函数
loss_fn = nn.CrossEntropyLoss()
# 定义优化器，优化模型上的所有参数和学习率，默认lr=1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# 如果GPUs可用，则将模型上需要计算的所有参数复制到GPUs上
if torch.cuda.is_available():
    model = model.cuda()
if torch.cuda.is_available():
    X, y = X.cuda(), y.cuda()

RuntimeError: CUDA error: out of memory

In [141]:
# 数据类实例
dst = Dataset()
# 模型类实例
model = VGG16()
# 配置类实例
conf = Config()

# 定义损失函数
loss_fn = nn.CrossEntropyLoss()
# 定义优化器，优化模型上的所有参数和学习率，默认lr=1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# 定义超级参数
epoch_n = 10

# 如果GPUs可用，则将模型上需要计算的所有参数复制到GPUs上
if torch.cuda.is_available():
    model = model.cuda()

# 代码执行时间装饰器
def timer(func):
    def wrapper(*args, **kw):
        begin = time.time()
        # 执行函数体
        func(*args, **kw)
        end = time.time()

        # 花费时间
        cost = end - begin
        print('本次一共花费时间：{:.2f}秒。'.format(cost))
    return wrapper


# 训练
@timer
def train():
    for epoch in range(1, epoch_n + 1):
        print('Epoch {}/{}'.format(epoch, epoch_n))
        print('-'*20)

        for phase in ['trainData', 'validData']:
            if phase == 'trainData':
                print('Training...')
                # 打开训练模式
                model.train(True)
            else:
                print('Validing...')
                # 关闭训练模式
                model.train(False)

            # 损失值
            running_loss = 0.0
            # 预测的正确数
            running_correct = 0
            # 让batch的值从1开始，便于后面计算
            for batch, data in enumerate(dst.data_images_loader[phase], 1):
                # 实际输入值和输出值
                X, y = data
                # 将参数复制到GPUs上进行运算
                if torch.cuda.is_available():
                    X, y = X.cuda(), y.cuda()
                # outputs.shape = [32,2] -> [1,2]
                outputs = model(X)
                # 从输出结果中取出需要的预测值
                # axis = 1 表示去按行取最大值
                _, y_pred = torch.max(outputs.detach(), 1)
                # 将累积的梯度置零
                optimizer.zero_grad()
                # 计算损失值
                loss = loss_fn(outputs, y)
                if phase == 'train':
                    # 反向传播求导
                    loss.backward()
                    # 更新所有参数
                    optimizer.step()

                running_loss += loss.detach().item()
                running_correct += torch.sum(y_pred == y)

                if batch % 500 == 0 and phase == 'train':
                    print('Batch {}/{},Train Loss:{:.2f},Train Acc:{:.2f}%'.format(
                        batch, len(dst.data_images[phase])/conf.batch_size, running_loss/batch, 100*running_correct.item()/(conf.batch_size*batch)
                    ))
            epoch_loss = running_loss*conf.batch_size/len(dst.data_images[phase])
            epoch_acc = 100*running_correct.item()/len(dst.data_images[phase])
            print('{} Loss:{:.2f} Acc:{:.2f}%'.format(phase, epoch_loss, epoch_acc))



AttributeError: 'Config' object has no attribute 'datas_root'